### 1 - Import test data

In [17]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df=df[df.High!=df.Low]
df.set_index("Gmt time", inplace=True)

In [18]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=7)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)
df

,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Gmt time,,,,,,,,,,,,,,
2019-09-30 00:00:00,1.09425,1.09426,1.09405,1.09406,585.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:05:00,1.09408,1.09414,1.09401,1.09409,289.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:10:00,1.09410,1.09423,1.09408,1.09410,276.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:15:00,1.09409,1.09410,1.09388,1.09389,439.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:20:00,1.09390,1.09395,1.09388,1.09395,341.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 20:35:00,0.98028,0.98034,0.98001,0.98022,624.12,0.979837,0.980175,42.710552,0.000423,0.979757,0.980317,0.980877,0.114260,0.413104
2022-09-30 20:40:00,0.98023,0.98047,0.98007,0.98030,408.20,0.979855,0.980183,45.967084,0.000420,0.979809,0.980344,0.980879,0.109099,0.458861
2022-09-30 20:45:00,0.98026,0.98034,0.98019,0.98031,317.29,0.979873,0.980191,46.411316,0.000381,0.979943,0.980391,0.980838,0.091333,0.409912


In [19]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 9999/9999 [00:20<00:00, 493.46it/s]
C:\Users\pierrek\AppData\Local\Temp\ipykernel_30556\3663350097.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<10
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>90
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 9999/9999 [00:40<00:00, 243.98it/s]
C:\Users\pierrek\AppData\Local\Temp\ipykernel_30556\309118033.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
df[df.TotalSignal != 0].head(20)

,Gmt time,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal,pointpos
3,2022-08-15 03:25:00,1.02478,1.02492,1.02474,1.02487,723.19,1.025308,1.025030,59.724787,0.000268,1.024246,1.024556,1.024866,0.060462,1.006890,1,1,1.02592
7,2022-08-15 03:45:00,1.02491,1.02523,1.02489,1.02508,569.41,1.025250,1.025005,68.397740,0.000257,1.024224,1.024621,1.025019,0.077612,1.076775,1,1,1.02623
8,2022-08-15 03:50:00,1.02508,1.02525,1.02504,1.02525,370.81,1.025250,1.025021,74.358910,0.000251,1.024225,1.024677,1.025129,0.088246,1.134054,1,1,1.02625
9,2022-08-15 03:55:00,1.02523,1.02533,1.02518,1.02523,591.27,1.025249,1.025034,72.482303,0.000236,1.024276,1.024741,1.025206,0.090773,1.026057,1,1,1.02633
11,2022-08-15 04:05:00,1.02519,1.02534,1.02507,1.02534,740.19,1.025250,1.025063,74.133640,0.000244,1.024458,1.024878,1.025298,0.081879,1.050553,1,1,1.02634
13,2022-08-15 04:15:00,1.02530,1.02552,1.02525,1.02549,366.86,1.025261,1.025104,76.568945,0.000237,1.024590,1.025001,1.025412,0.080210,1.094371,1,1,1.02652
14,2022-08-15 04:20:00,1.02549,1.02559,1.02544,1.02549,217.41,1.025270,1.025129,76.568945,0.000225,1.024660,1.025065,1.025469,0.078979,1.025374,1,1,1.02659
16,2022-08-15 04:30:00,1.02533,1.02558,1.02531,1.02555,644.86,1.025283,1.025168,70.101680,0.000228,1.024781,1.025163,1.025545,0.074544,1.005976,1,1,1.02658
53,2022-08-15 07:35:00,1.02405,1.02446,1.02395,1.02415,1615.68,1.024400,1.024132,57.739378,0.000575,1.023421,1.023782,1.024143,0.070613,1.009041,1,1,1.02546
112,2022-08-15 12:30:00,1.02026,1.02149,1.02026,1.02095,3551.63,1.020541,1.020083,75.917402,0.000664,1.019117,1.019821,1.020526,0.138127,1.301238,1,1,1.02249


In [33]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

C:\Users\pierrek\AppData\Local\Temp\ipykernel_30556\2345118794.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=2033
dfpl = df[st:]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL(Bollinger Band Lower)"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU(Bollinger Band Upper)"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [35]:
def SIGNAL():
    return df.TotalSignal

In [36]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)

C:\Users\pierrek\AppData\Local\Temp\ipykernel_30556\2590162779.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [37]:
bt.run()

Start                                     0.0
End                                    9998.0
Duration                               9998.0
Exposure Time [%]                    31.92319
Equity Final [$]                    399.68144
Equity Peak [$]                     404.31403
Return [%]                           59.87258
Buy & Hold Return [%]                -4.36416
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -11.22854
Avg. Drawdown [%]                    -1.38015
Max. Drawdown Duration                 2324.0
Avg. Drawdown Duration              104.64198
# Trades                                560.0
Win Rate [%]                         44.64286
Best Trade [%]                         0.3773
Worst Trade [%]                      -0.25218
Avg. Trade [%]                    

In [12]:
bt.plot()

GridPlot(id='p1346', ...)